In [1]:
import os
import json
from datetime import datetime as dt
import pandas as pd

In [2]:
# modificar segun entorno local
os.chdir( "/Volumes/hd_mvf_datasets/henry_data")

# no modificar
folder_data = "1_external"
folder_pipeline = "2_pipeline"
folder_output = "3_output"
folder_gmaps = "gmaps"
folder_reviews = "reviews_estados"

In [3]:
# extraemos nombres de cada estado segun nombre de carpeta
state_name = pd.DataFrame(
    pd.DataFrame(
        os.listdir(
            os.path.join(folder_data,folder_gmaps,folder_reviews))).rename(
                columns={0:"state"})["state"].str.split("-").str[1])
state_name.head()

,state
0,California
1,Pennsylvania


In [4]:
# indentificamos las columnas de interes y aquellas que se deben desanidar
vars_interes=[
    "gmap_id" # pk google maps
    ,"user_id" # pk user >> resp
    ,"name"
    ,"time"
    ,"rating"
    ,"text"
    #,"pics" # irrelevante
    #,"resp" # desanidar
    ]

vars_desanidar=[
    "gmap_id" # pk google maps
    ,"user_id" # pk user >> resp
    ,"time" # pk user >> resp
    ,"resp" # desanidar
    ]

In [5]:
for i in range(min(state_name.index),max(state_name.index)):

    # creamos una variable con el nombre de estado
    # para facilitar el loop para todos los estados
    state = state_name["state"][i]
    folder_state = "review-" + str(state)

    # contamos numero de archivos al interior de cada carpeta del estado
    # para facilitar el loop
    count_file = 0
    # Iterate
    for path in os.listdir(os.path.join(folder_data,folder_gmaps,folder_reviews,folder_state)):
        # check if current path is a file
        if os.path.isfile(os.path.join(folder_data,folder_gmaps,folder_reviews,folder_state,path)):
            count_file += 1

    # extraemos la informacion
    data = []
    for f in range(1, count_file):
        with open(os.path.join(
            folder_data,folder_gmaps,folder_reviews,folder_state, str(f) + ".json"), "r") as file:
            for line in file:
                data.append(json.loads(line))
    gmaps_state = pd.DataFrame(data)

    # nos concentramos en: vars_desanidar
    gmaps_state_dsndr = pd.DataFrame(gmaps_state[vars_desanidar]).dropna(subset=["gmap_id","user_id"])
    gmaps_state_dsndr.rename(
        columns={"time":"user_time"},
        inplace=True
        )

    # se normaliza la columna
    gmaps_state_dsndr = gmaps_state_dsndr.set_index(["gmap_id","user_id","user_time"])
    gmaps_state_dsndda = pd.json_normalize(
        gmaps_state_dsndr["resp"]).set_index(gmaps_state_dsndr.index) # type: ignore
    gmaps_state_dsndda.reset_index(inplace=True) # mueve le indice a una columna
    # renombramos para facilitar la union mas abajo
    gmaps_state_dsndda.rename(
        columns={"time":"resp_time","text":"resp_text"},
        inplace=True
        )
    # solo incluimos las variables de interes
    gmaps_state_interes = gmaps_state[vars_interes].copy().dropna(
        subset=["gmap_id","user_id"]) # type: ignore
    # renombramos para facilitar la union mas abajo
    gmaps_state_interes.rename(
        columns={"name":"user_name","time":"user_time","text":"user_text"},
        inplace=True
        )

    # unir los dataframes
    gmaps_state_norm = pd.merge(
        gmaps_state_interes,
        gmaps_state_dsndda,
        on=["gmap_id","user_id","user_time"],
        how="left")

    # generamos una columna state
    # para facilitar la union de todos los datos
    gmaps_state_norm["state"] = state

    # movemos 'state' a la primera fila para facilitar la visualizacion
    first_column = gmaps_state_norm.pop("state")
    gmaps_state_norm.insert(0, "state", first_column)
    
    # Convierte la columna 'user_time' a datetime y almacena el resultado en una nueva columna 'time_total'
    gmaps_state_norm['user_time_year'] = pd.to_datetime(gmaps_state_norm['user_time'], unit='ms').dt.year # type: ignore
    gmaps_state_norm['user_time_month'] = pd.to_datetime(gmaps_state_norm['user_time'], unit='ms').dt.month # type: ignore
    gmaps_state_norm['user_time_day'] = pd.to_datetime(gmaps_state_norm['user_time'], unit='ms').dt.day # type: ignore
    gmaps_state_norm['user_time_hms'] = pd.to_datetime(gmaps_state_norm['user_time'], unit='ms').dt.time # type: ignore
    gmaps_state_norm['resp_time_year'] = pd.to_datetime(gmaps_state_norm['resp_time'], unit='ms').dt.year # type: ignore
    gmaps_state_norm['resp_time_month'] = pd.to_datetime(gmaps_state_norm['resp_time'], unit='ms').dt.month # type: ignore
    gmaps_state_norm['resp_time_day'] = pd.to_datetime(gmaps_state_norm['resp_time'], unit='ms').dt.day # type: ignore
    gmaps_state_norm['resp_time_hms'] = pd.to_datetime(gmaps_state_norm['resp_time'], unit='ms').dt.time # type: ignore

    # Removemos duplicados
    gmaps_state_norm.drop_duplicates(inplace=True)
    
    # filtramos valores post-pandamie 1 julio de 2020
    gmaps_state_norm_filtrada = gmaps_state_norm.loc[
        (gmaps_state_norm['user_time_year'] >= 2020) &
        (gmaps_state_norm['user_time_month'] >= 7)
        ].copy()
    
    # df final por estado
    # guardamos en pipeline con el fin de alivianar la carga al RAM local
    gmaps_state_norm_filtrada.to_parquet(
        os.path.join(folder_pipeline,folder_gmaps, str(folder_state) + "_norm.parquet"))

In [6]:
gmaps_state_norm_filtrada[gmaps_state_norm_filtrada.duplicated(keep=False)]

,state,gmap_id,user_id,user_name,user_time,rating,user_text,resp_time,resp_text,user_time_year,user_time_month,user_time_day,user_time_hms,resp_time_year,resp_time_month,resp_time_day,resp_time_hms


In [7]:
# ultimo estado;
# df debe ser igual a archivo de etl_1
gmaps_state_norm_filtrada.info()

<class 'pandas.core.frame.DataFrame'>
Index: 258984 entries, 16 to 2687477
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   state            258984 non-null  object 
 1   gmap_id          258984 non-null  object 
 2   user_id          258984 non-null  object 
 3   user_name        258984 non-null  object 
 4   user_time        258984 non-null  int64  
 5   rating           258984 non-null  int64  
 6   user_text        151881 non-null  object 
 7   resp_time        34984 non-null   float64
 8   resp_text        34984 non-null   object 
 9   user_time_year   258984 non-null  int32  
 10  user_time_month  258984 non-null  int32  
 11  user_time_day    258984 non-null  int32  
 12  user_time_hms    258984 non-null  object 
 13  resp_time_year   34984 non-null   float64
 14  resp_time_month  34984 non-null   float64
 15  resp_time_day    34984 non-null   float64
 16  resp_time_hms    34984 non-null   object 